In [1]:
import json
import pandas as pd
import elasticsearch
from groq import Groq

from dotenv import load_dotenv
import os
load_dotenv()
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
import os
os.getcwd()

'd:\\Projects\\AI-Restaurent-Chat-bot\\notebooks'

In [3]:
# change path for base folder
base_folder = 'D:/Projects/AI-Restaurent-Chat-bot/'
input_data_folder = base_folder+'input_data/'

In [1]:
# Setup the OpenAI client to use either Groq, OpenAI.com, or Ollama API
load_dotenv(override=True)
API_HOST = os.getenv("API_HOST")
API_HOST

NameError: name 'load_dotenv' is not defined

In [ ]:
if API_HOST == "groq":
    client = Groq(api_key=os.getenv('GROQ_API_KEY'))
    MODEL_NAME = os.getenv("GROQ_MODEL")

elif API_HOST == "ollama":
    client = openai.OpenAI(
        base_url=os.getenv("OLLAMA_ENDPOINT"),
        api_key="nokeyneeded",
    )
    MODEL_NAME = os.getenv("OLLAMA_MODEL")

elif API_HOST == "github":
    client = openai.OpenAI(base_url="https://models.inference.ai.azure.com", api_key=os.getenv("GITHUB_TOKEN"))
    MODEL_NAME = os.getenv("GITHUB_MODEL")

else:
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    MODEL_NAME = os.getenv("OPENAI_MODEL")

In [4]:
# openai api
from openai import OpenAI
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
with open(input_data_folder + 'food_user_qa_dataset.json', 'rt') as f_in:
    data = json.load(f_in)

In [6]:
documents = []
for dish in data['dishes']:
    dish_name = dish['dish name']
    for doc in dish['documents']:
        doc['dish_name'] = dish_name  # Add dish_name to each document
        documents.append(doc)

In [8]:
documents[1:10]

[{'id': '142_2',
  'question': 'How many calories does it have?',
  'section': 'calories',
  'text': 'The almond fudge banana cake has 224.8 calories.',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_3',
  'question': 'How much total fat does it contain?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 14 grams of total fat (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_4',
  'question': 'What is the sugar content?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 87 grams of sugar (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_5',
  'question': 'How much sodium is in this dish?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 10 grams of sodium (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_6',
  'question': 'What is the protein content?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 7 grams of protein (PDV).',
  'dish_

In [8]:
import wget
url= "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"
#wget.download(url)



In [9]:
import minsearch

In [10]:
index = minsearch.Index(
    text_fields = ['id', 'question','section','text','dish_name'],
    keyword_fields=['dish_name']
    #keyword_fields=['id']

)

In [11]:
index.fit(documents)

In [12]:
question ="pistachio cake  from mix"

In [ ]:
question ="pistachio cake  from mix"

In [13]:
def minsearch(question):
    return index.search(question)

In [14]:
def elastic_search(query, dish_name=None):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["dish_name^6", "question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    }
                ]
            }
        }
    }

    # Add a filter by dish_name if it's provided
    if dish_name:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "dish_name.keyword": dish_name
            }
        }

    # Perform the search
    response = es.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)  # Collect all documents
        

    return result_docs


In [15]:
search_results= minsearch(question)
search_results

[{'id': '15664_12',
  'question': 'What is the rating of this dish?',
  'section': 'rating',
  'text': 'The pistachio cake  from mix has a rating of 5.',
  'dish_name': 'pistachio cake  from mix'},
 {'id': '15664_6',
  'question': 'What is the protein content?',
  'section': 'nutritional',
  'text': 'The pistachio cake  from mix has 6 grams of protein (PDV).',
  'dish_name': 'pistachio cake  from mix'},
 {'id': '15664_5',
  'question': 'How much sodium is in this dish?',
  'section': 'nutritional',
  'text': 'The pistachio cake  from mix has 9 grams of sodium (PDV).',
  'dish_name': 'pistachio cake  from mix'},
 {'id': '15664_8',
  'question': 'What are the carbohydrate levels?',
  'section': 'nutritional',
  'text': 'The pistachio cake  from mix has 8 grams of carbohydrates (PDV).',
  'dish_name': 'pistachio cake  from mix'},
 {'id': '15664_7',
  'question': 'How much saturated fat is present?',
  'section': 'nutritional',
  'text': 'The pistachio cake  from mix has 9 grams of saturat

In [21]:
documents[1]

{'id': '142_2',
 'question': 'How many calories does it have?',
 'section': 'calories',
 'text': 'The almond fudge banana cake has 224.8 calories.',
 'dish_name': 'almond fudge banana cake'}

In [22]:
# Converting our database into vectors.

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\nakka\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nakka\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

c:\Users\nakka\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
model.encode("abhishek")

array([ 1.45805469e-02, -1.99083053e-03, -2.13489053e-03, -1.80335604e-02,
       -5.10015637e-02, -1.65787507e-02, -1.78630021e-03,  1.76838841e-02,
        6.66056052e-02, -1.00949463e-02,  3.96251082e-02, -5.45170754e-02,
        3.80212590e-02,  4.14985642e-02,  1.05339047e-02, -3.68569940e-02,
       -2.41204258e-02, -1.19364923e-02, -2.67536901e-02, -5.52969053e-03,
       -4.20143865e-02,  5.30301081e-03, -5.64692095e-02,  3.84490602e-02,
        5.69248525e-03, -2.42834515e-03,  9.71981697e-03, -1.39138559e-02,
        4.13596956e-03,  1.58024840e-02,  2.86336299e-02, -3.80830690e-02,
       -4.45273370e-02,  5.28516769e-02,  1.83281099e-06,  1.30049260e-02,
        1.69290733e-02, -4.20571043e-04,  7.40678376e-03, -3.86865214e-02,
        5.47060035e-02,  8.54937509e-02, -2.05739923e-02, -2.62397584e-02,
        1.36871298e-04,  3.70169058e-02,  2.27064602e-02, -1.52531080e-02,
       -2.74782162e-02,  1.20341603e-03,  6.66463189e-03, -4.47323844e-02,
       -4.63145338e-02,  

In [24]:
documents[1]

{'id': '142_2',
 'question': 'How many calories does it have?',
 'section': 'calories',
 'text': 'The almond fudge banana cake has 224.8 calories.',
 'dish_name': 'almond fudge banana cake'}

In [25]:
operations = []

for doc in documents:
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [ ]:
# user question into embeddings

search_term = "Windows or mac?"
vector_search_term = model.encode(search_term)

In [ ]:
query = {
    "field":"text_vector",
    "query_vector": vector_search_term,
    "k":5,
    "num_candidates":10000,
}

In [ ]:
# pass query to elastic search



In [16]:
# if using OPENAI 
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [17]:
# iF USING groq
"""
from groq import Groq   

#create client calling Groq class

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

"""

'\nfrom groq import Groq   \n\n#create client calling Groq class\n\nclient = Groq(api_key=os.getenv(\'GROQ_API_KEY\'))\n\nfrom openai import OpenAI\nclient = OpenAI()\n\ndef llm(prompt):\n    response = client.chat.completions.create(\n        model="llama3-8b-8192",\n        messages=[{"role": "user", "content": prompt}]\n    )\n    \n    return response.choices[0].message.content\n\n'

In [18]:
def build_prompt(query, search_results):
    prompt_template = """
You're an AI assistant helping with menu queries. Answer the QUESTION based on the CONTEXT provided.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [19]:
def rag(query):
    search_results = minsearch(query)
    prompt=build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [20]:
query = "what are the calories of white chocolate cake is it good?"
rag(query)

'Based on the context provided, the white chocolate cake has 314.9 calories.'